## 1. 1D Kalman Filter – Base Setup

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
dt = 1.0
n_steps = 50
true_velocity = 1.0
measurement_noise_std = 2.0

true_positions = []
measurements = []
pos = 0.0

for _ in range(n_steps):
    pos += true_velocity * dt
    true_positions.append(pos)
    measurements.append(pos + np.random.normal(0, measurement_noise_std))

true_positions = np.array(true_positions)
measurements = np.array(measurements)


## 2. 1D Kalman Filter Implementation

In [ ]:

def kalman_filter_1d(Q_val=0.1, R_val=4.0, P0=10.0, x0=0.0):
    x = np.array([[x0]])
    P = np.array([[P0]])
    F = np.array([[1.0]])
    H = np.array([[1.0]])
    Q = np.array([[Q_val]])
    R = np.array([[R_val]])
    
    estimates = []
    for z in measurements:
        x_pred = F @ x
        P_pred = F @ P @ F.T + Q
        
        K = P_pred @ H.T @ np.linalg.inv(H @ P_pred @ H.T + R)
        x = x_pred + K @ (np.array([[z]]) - H @ x_pred)
        P = (np.eye(1) - K @ H) @ P_pred
        
        estimates.append(x.item())
        
    return np.array(estimates)


## 3. Parameter Exploration

In [ ]:

est_high_R = kalman_filter_1d(R_val=25.0)
est_high_Q = kalman_filter_1d(Q_val=2.0)
est_small_P = kalman_filter_1d(P0=0.01)
est_wrong_x0 = kalman_filter_1d(x0=50.0)

plt.figure()
plt.plot(true_positions, label="True")
plt.plot(measurements, linestyle="dotted", label="Measurements")
plt.plot(est_high_R, label="High R")
plt.plot(est_high_Q, label="High Q")
plt.legend()
plt.title("Effect of R and Q")
plt.show()

plt.figure()
plt.plot(true_positions, label="True")
plt.plot(est_small_P, label="Small Initial P")
plt.plot(est_wrong_x0, label="Wrong Initial State")
plt.legend()
plt.title("Effect of Initial Conditions")
plt.show()


## 4. Extension – Position + Velocity Kalman Filter

In [ ]:

true_pos, true_vel = 0.0, 0.5
acc_std = 0.2

true_states = []
measurements = []

for _ in range(n_steps):
    acc = np.random.normal(0, acc_std)
    true_pos += true_vel * dt + 0.5 * acc * dt**2
    true_vel += acc * dt
    true_states.append([true_pos, true_vel])
    measurements.append(true_pos + np.random.normal(0, measurement_noise_std))

true_states = np.array(true_states)
measurements = np.array(measurements)

x = np.array([[0.0],[0.0]])
P = np.eye(2)*10
F = np.array([[1, dt],[0,1]])
H = np.array([[1,0]])
Q = np.eye(2)*0.1
R = np.array([[measurement_noise_std**2]])

kf_pos = []

for z in measurements:
    x_pred = F @ x
    P_pred = F @ P @ F.T + Q
    K = P_pred @ H.T @ np.linalg.inv(H @ P_pred @ H.T + R)
    x = x_pred + K @ (np.array([[z]]) - H @ x_pred)
    P = (np.eye(2) - K @ H) @ P_pred
    kf_pos.append(x[0,0])

plt.figure()
plt.plot(true_states[:,0], label="True Position")
plt.plot(measurements, linestyle="dotted", label="Measurements")
plt.plot(kf_pos, label="KF Estimate")
plt.legend()
plt.title("Position + Velocity Kalman Filter")
plt.show()


## 5. Conclusion

Kalman Filter successfully balances model prediction and noisy measurements using uncertainty. Parameter tuning of Q, R, and P strongly affects convergence and smoothness.